## Cylindre

In [4]:
import pyvista as pv
import dolfinx
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np
import gmsh
import meshio
from dolfinx.io import XDMFFile

import dolfinx.fem as fem
import dolfinx.mesh as mesh
from petsc4py import PETSc

# Initialisation de GMSH
gmsh.initialize()
gmsh.model.add("ellipse")

# Définition des variables
a = 0.2  # Demi-grand axe en mètres
b = 0.2  # Demi-petit axe en mètres
h = 1.0  # Hauteur du cylindre en mètres
lc = 0.02  # Longueur caractéristique des éléments en mètres

# Créer une ellipse dans le plan XY au centre (0, 0, 0)
ellipse = gmsh.model.occ.addEllipse(0, 0, 0, a, b)

# Créer une boucle fermée autour de l'ellipse
curve_loop = gmsh.model.occ.addCurveLoop([ellipse])

# Créer une surface à partir de la boucle
surface = gmsh.model.occ.addPlaneSurface([curve_loop])

# Extruder cette surface pour créer un cylindre elliptique de hauteur h
gmsh.model.occ.extrude([(2, surface)], 0, 0, h)

# Synchronisation de la géométrie
gmsh.model.occ.synchronize()

# Définir les tailles des éléments
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)

# Générer le maillage
gmsh.model.mesh.generate(3)  # 3 pour le maillage en 3D

# Sauvegarder le maillage dans un fichier .msh
gmsh.write("ellipse.msh")

# Finaliser GMSH
gmsh.finalize()

# Lire le fichier .msh avec meshio
msh = meshio.read("ellipse.msh")

# Sauvegarder le maillage en format XDMF
meshio.write("ellipse.xdmf", meshio.Mesh(points=msh.points, cells={"tetra": msh.cells_dict.get("tetra", [])}))

# Lire le fichier .xdmf avec Dolfinx
with XDMFFile(MPI.COMM_WORLD, "ellipse.xdmf", "r") as xdmf_file:
    domain = xdmf_file.read_mesh(name="Grid")
    domain.topology.create_connectivity(domain.topology.dim-1, domain.topology.dim)

# Afficher des informations sur le maillage
print("Maillage créé avec succès.")
print(f"Nombre de sommets : {domain.topology.index_map(domain.topology.dim).size_global}")
print(f"Nombre d'éléments : {domain.topology.index_map(domain.topology.dim-1).size_global}")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Ellipse)
Info    : Done meshing 1D (Wall 0.000237999s, CPU 0.000317s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Unknown, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.19491s, CPU 0.193173s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 4451 nodes...
Info    : Done tetrahedrizing 4459 nodes (Wall 0.0351201s, CPU 0.034297s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.168512s, CPU 0.165074s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 10.6389 (nodes removed 0 0)
Info    : It. 5

In [5]:
# Création du maillage pour PyVista basé sur les coordonnées des dofs
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(domain)

# Créez la grille PyVista et ajoutez les valeurs des dofs à la grille
u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)

# Visualisation
p = pv.Plotter()

#u_grid.point_data["u"] = uh.x.array.reshape((u_geometry.shape[0], 2))
#warped = u_grid.warp_by_scalar("u", factor=10)
p.add_mesh(u_grid, show_edges=True, scalar_bar_args={
    "title": "u",
    "title_font_size": 24,
    "label_font_size": 22,
    "shadow": True,
    "italic": True,
    "font_family": "arial",
    "vertical": False
})
p.add_text("Cylindre Mesh", font_size=12, color="black", position="upper_edge")
p.show_bounds(color="black")
p.add_axes(color="black")
p.set_background("grey")
p.show()
p.view_yz()

Widget(value='<iframe src="http://localhost:51527/index.html?ui=P_0x104095b80_1&reconnect=auto" class="pyvista…

In [6]:
R=1/2*np.sqrt(a**2+b**2)

V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))

def down_face(x):
    return np.isclose(x[2], 0) 

def top_face(x):
    return np.isclose(x[2], h)  
    
def lateral_face(x):
    tolerance = 1e-5  # Tolérance pour la comparaison
    return (np.isclose((x[0]**2 / a**2 + x[1]**2 / b**2), 1.0, atol=tolerance)) & (0 <= x[2]) & (x[2] <= h)
    
fdim = domain.topology.dim - 1

Sigma_l = mesh.locate_entities(domain, fdim, lateral_face)
Sigma_0 = mesh.locate_entities(domain, fdim, down_face)
Sigma_h = mesh.locate_entities(domain, fdim, top_face)

u_D = np.array([0, 0, 0], dtype=default_scalar_type)

bc1 = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, Sigma_0), V)
bc2 = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, Sigma_h), V)
bc3 = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, Sigma_l), V)

In [7]:
import numpy as np

# Définir une fonction pour appliquer des marqueurs
def apply_marker(boundary_facets, marker_array, marker_value):
    boundary_facets = boundary_facets[boundary_facets < num_cells_local]
    marker_array[boundary_facets] = marker_value

# Obtenir le nombre de cellules locales
num_cells_local = domain.topology.index_map(fdim).size_local

# Initialiser les tableaux de marqueurs
markers = [np.zeros(num_cells_local, dtype=np.int32) for _ in range(4)]

# Appliquer les marqueurs
apply_marker(Sigma_0, markers[0], 1)  # Marqueur 1
apply_marker(Sigma_h, markers[1], 2)  # Marqueur 2
apply_marker(Sigma_l, markers[2], 3)  # Marqueur 3

# Calculer le marqueur combiné
markers[3] = markers[0] + markers[1] + markers[2]

# Ajouter un marqueur spécifique pour les cellules restantes (si nécessaire)
markers[3][markers[3] == 0] = 4

# Créer la connectivité du domaine
domain.topology.create_connectivity(fdim, fdim)

# Obtenir les données de maillage pour PyVista
topology, cell_types, x = dolfinx.plot.vtk_mesh(domain, fdim, np.arange(num_cells_local, dtype=np.int32))

In [8]:
import pyvista as pv
import numpy as np

# Création du maillage PyVista
grid = pv.UnstructuredGrid(topology, cell_types, x)

# Fonction pour ajouter des maillages à une fenêtre
def add_plot(ax, marker, color, title, threshold_min):
    # Mettre à jour les données de cellule du maillage
    grid.cell_data["Marker"] = marker
    grid.set_active_scalars("Marker")
    
    # Ajouter le maillage avec les marqueurs
    ax.add_mesh(grid, show_edges=True, color="cyan", scalar_bar_args={
        "title": "Boundary Marker",
        "title_font_size": 24,
        "label_font_size": 22,
        "shadow": True,
        "italic": True,
        "font_family": "arial",
        "vertical": False
    })
    
    # Appliquer un filtre basé sur le seuil
    grid_filter = grid.threshold(threshold_min, scalars='Marker')
    ax.add_mesh(grid_filter, color=color, show_edges=True)
    
    # Ajouter le texte et les axes
    ax.add_text(title, font_size=12, color="black", position="upper_edge")
    ax.add_axes(color="black")

# Visualisation avec PyVista
pl = pv.Plotter(shape=(2, 2))

# Fenêtre 1 : Afficher toutes les cellules
pl.subplot(0, 0)
add_plot(pl, markers[3], "red", "All Boundaries", threshold_min=0.5)

# Fenêtre 2 : Afficher les cellules avec marker1
pl.subplot(0, 1)
add_plot(pl, markers[0], "red", "Down Boundary", threshold_min=0.5)

# Fenêtre 3 : Afficher les cellules avec marker2
pl.subplot(1, 0)
add_plot(pl, markers[1], "red", "Top Boundary", threshold_min=1.5)

# Fenêtre 4 : Afficher les cellules avec marker3
pl.subplot(1, 1)
add_plot(pl, markers[2], "red", "Lateral Boundary", threshold_min=2.5)

# Configuration finale
pl.set_background("grey")
pl.show()

Widget(value='<iframe src="http://localhost:51527/index.html?ui=P_0x164f4ef90_2&reconnect=auto" class="pyvista…

In [12]:
mu = 80e9
lambda_ = 120e9

import math
pi = math.pi  # Pi
omega = 20  # Angle en radians
# Calcul d'alpha
alpha = omega / h
print(f"α = {alpha} m^-1")
# Calcul de C
C = mu * pi * (R ** 4) / 2 * alpha
print(f"C = {C} kg.m^2/s")

α = 20.0 m^-1
C = 1005309649.1487346 kg.m^2/s


In [13]:
A=2 * C / (pi * R**4) # A un scalaire constant positif
x = ufl.SpatialCoordinate(domain)
# Définition des composantes de torsion
T1 = -A * x[1] 
T2 = A * x[0]
T3 = ufl.as_ufl(0.0)  # T3 = 0, en tant que constante UFL
# Combinaison en un champ vectoriel
T = ufl.as_vector([T1, T2, T3])

#T = fem.Constant(domain, default_scalar_type([0, 0, 10]))

# Marqueurs des facettes
marked_facets = np.hstack([Sigma_h])
#marked_values = np.hstack([np.full_like(Sigma_0, 1), np.full_like(Sigma_h, 2)])
marked_values = np.hstack([np.full_like(Sigma_h, 1)])
# Tri des facettes et valeurs
sorted_facets = np.argsort(marked_facets)
facet_tag = dolfinx.mesh.meshtags(domain, domain.topology.dim - 1, marked_facets[sorted_facets], marked_values[sorted_facets])

# Définir une mesure pour les facettes marquées
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag)

#ufl.dot(M, ufl.cross(ufl.grad(u), v)) 

def epsilon(u):
    return ufl.sym(ufl.grad(u))  # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T) ou 0.5 * (ufl.grad(u).T + ufl.grad(u))

def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Extraire les composantes du gradient et les assembler en un vecteur
grad_u = ufl.grad(u)
grad_u_vec = ufl.as_vector([grad_u[0, 0], grad_u[1, 1], grad_u[2, 2]]) 
product_vectoriel = ufl.cross(v, grad_u_vec)

f = fem.Constant(domain, default_scalar_type((0, 0, 0)))
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds(1)  # Terme de force volumique, pression et moment de torsion
problem = LinearProblem(a, L, bcs=[bc1], petsc_options={"ksp_type": "cg", "pc_type": "jacobi"})
#problem = LinearProblem(a, L, bcs=[bc1], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [14]:
print(sigma)

<function sigma at 0x163a56020>


In [15]:
# Création du maillage pour PyVista basé sur les coordonnées des dofs
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(domain)

# Créez la grille PyVista et ajoutez les valeurs des dofs à la grille
u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)

# Convertir le vecteur de solution en format compatible pour PyVista
#uh_vectors = uh.x.array.reshape((-1, domain.geometry.dim))

# Attach vector values to grid and warp grid by vector
u_grid["u"] = uh.x.array.reshape((u_geometry.shape[0], 3))

# Visualisation
p = pv.Plotter()
p.add_mesh(u_grid, show_edges=True, scalar_bar_args={
    "title": "u",
    "title_font_size": 24,
    "label_font_size": 22,
    "shadow": True,
    "italic": True,
    "font_family": "arial",
    "vertical": False
})
p.add_text("Déplacements", font_size=12, color="black", position="upper_edge")
p.add_axes(color="black")
p.set_background("grey")
p.show()

Widget(value='<iframe src="http://localhost:51527/index.html?ui=P_0x16c180920_3&reconnect=auto" class="pyvista…

In [16]:
dargs = dict(
    scalars="u",
    cmap="jet",
    show_scalar_bar=False,
)

pl = pv.Plotter(shape=(2, 2))
pl.subplot(0, 0)
pl.add_mesh(u_grid, **dargs)
pl.add_axes(color="black")
pl.add_text("Normalized Displacement", color='k')
pl.subplot(0, 1)
pl.add_mesh(u_grid.copy(), component=0, **dargs)
pl.add_axes(color="black")
pl.add_text("X Displacement", color='k')
pl.subplot(1, 0)
pl.add_mesh(u_grid.copy(), component=1, **dargs)
pl.add_axes(color="black")
pl.add_text("Y Displacement", color='k')
pl.subplot(1, 1)
pl.add_mesh(u_grid.copy(), component=2, **dargs)
pl.add_axes(color="black")
pl.add_text("Z Displacement", color='k')


# Lier les vues pour une vue cohérente
pl.link_views()
pl.camera_position = 'iso'
pl.background_color = 'grey'
# Afficher le tout
pl.show()

Widget(value='<iframe src="http://localhost:51527/index.html?ui=P_0x16c18f2f0_4&reconnect=auto" class="pyvista…

In [17]:
###### Visualisation
p = pv.Plotter()
warped = u_grid.warp_by_vector("u", factor=0.04)

p.add_mesh(warped, show_edges=True, scalar_bar_args={
    "title": "u",
    "title_font_size": 24,
    "label_font_size": 22,
    "shadow": True,
    "italic": True,
    "font_family": "arial",
    "vertical": False
})
p.add_text("Déplacements", font_size=12, color="black", position="upper_edge")
p.add_axes(color="black")
p.set_background("grey")
p.show()

Widget(value='<iframe src="http://localhost:51527/index.html?ui=P_0x16c182bd0_5&reconnect=auto" class="pyvista…

In [18]:
import pyvista as pv
import numpy as np
import dolfinx
import ufl
from petsc4py import PETSc
from mpi4py import MPI
import matplotlib.pyplot as plt

# Calculer le tenseur de déformation
eps = epsilon(uh)

# Extraire les composantes du tenseur de déformation
eps_xx = eps[0, 0]
eps_xy = eps[0, 1]
eps_xz = eps[0, 2]
eps_yy = eps[1, 1]
eps_yz = eps[1, 2]
eps_zz = eps[2, 2]

# Créez la grille PyVista
eps_topology, eps_cell_types, eps_geometry = dolfinx.plot.vtk_mesh(domain)
eps_grid = pv.UnstructuredGrid(eps_topology, eps_cell_types, eps_geometry)

V_eps = fem.functionspace(domain, ("DG", 0))

eps_xx_expr = fem.Expression(eps_xx, V_eps.element.interpolation_points())
strain_xx = fem.Function(V_eps)
strain_xx.interpolate(eps_xx_expr)

eps_yy_expr = fem.Expression(eps_yy, V_eps.element.interpolation_points())
strain_yy = fem.Function(V_eps)
strain_yy.interpolate(eps_yy_expr)

eps_zz_expr = fem.Expression(eps_zz, V_eps.element.interpolation_points())
strain_zz = fem.Function(V_eps)
strain_zz.interpolate(eps_zz_expr)

eps_yz_expr = fem.Expression(eps_yz, V_eps.element.interpolation_points())
strain_yz = fem.Function(V_eps)
strain_yz.interpolate(eps_yz_expr)

eps_xz_expr = fem.Expression(eps_xz, V_eps.element.interpolation_points())
strain_xz = fem.Function(V_eps)
strain_xz.interpolate(eps_xz_expr)

eps_xy_expr = fem.Expression(eps_xy, V_eps.element.interpolation_points())
strain_xy = fem.Function(V_eps)
strain_xy.interpolate(eps_xy_expr)

pl = pv.Plotter(shape=(2, 3))

warped = u_grid.warp_by_vector("u", factor=0)

dargs = dict(
    cmap="jet",
    show_scalar_bar=False,
)

pl.subplot(0, 0)
warped.cell_data["Epsilon_XX"] = strain_xx.vector.array
warped.set_active_scalars("Epsilon_XX")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Epsilon XX", color='k')

pl.subplot(0, 1)
warped.cell_data["Epsilon_YY"] = strain_yy.vector.array
warped.set_active_scalars("Epsilon_YY")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Epsilon YY", color='k')

pl.subplot(0, 2)
warped.cell_data["Epsilon_ZZ"] = strain_zz.vector.array
warped.set_active_scalars("Epsilon_ZZ")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Epsilon ZZ", color='k')

pl.subplot(1, 0)
warped.cell_data["Epsilon_YZ"] = strain_yz.vector.array
warped.set_active_scalars("Epsilon_YZ")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Epsilon YZ", color='k')

pl.subplot(1, 1)
warped.cell_data["Epsilon_XZ"] = strain_xz.vector.array
warped.set_active_scalars("Epsilon_XZ")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Epsilon XZ", color='k')

pl.subplot(1, 2)
warped.cell_data["Epsilon_XY"] = strain_xy.vector.array
warped.set_active_scalars("Epsilon_XY")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Epsilon XY", color='k')

# Lier les vues pour une vue cohérente
pl.link_views()
pl.camera_position = 'iso'
pl.background_color = 'grey'
# Afficher le tout
pl.show()

Widget(value='<iframe src="http://localhost:51527/index.html?ui=P_0x2aecf9ca0_6&reconnect=auto" class="pyvista…

In [19]:
import pyvista as pv
import numpy as np
import dolfinx
import ufl
from petsc4py import PETSc
from mpi4py import MPI
import matplotlib.pyplot as plt

def omega(u):
    return 0.5 * (ufl.grad(u).T - ufl.grad(u))

theta = omega(uh)

theta_xx = theta[0, 0]
theta_xy = theta[0, 1]
theta_xz = theta[0, 2]
theta_yy = theta[1, 1]
theta_yz = theta[1, 2]
theta_zz = theta[2, 2]

V_omega = fem.functionspace(domain, ("DG", 0))

rotation_expr = fem.Expression(theta_yz, V_omega.element.interpolation_points())
rotation_x = fem.Function(V_omega)
rotation_x.interpolate(rotation_expr)

rotation_expr = fem.Expression(theta_xz, V_omega.element.interpolation_points())
rotation_y = fem.Function(V_omega)
rotation_y.interpolate(rotation_expr)

rotation_expr = fem.Expression(theta_xy, V_omega.element.interpolation_points())
rotation_z = fem.Function(V_omega)
rotation_z.interpolate(rotation_expr)

# Calculer la norme de rotation
norm_theta = ufl.sqrt(rotation_z**2 + rotation_y**2 + rotation_x**2)

rotation_expr = fem.Expression(norm_theta, V_omega.element.interpolation_points())
rotation_norm= fem.Function(V_omega)
rotation_norm.interpolate(rotation_expr)


pl = pv.Plotter(shape=(2, 2))
dargs = dict(
    cmap="jet",
    show_scalar_bar=False,
)
pl.subplot(0, 0)
warped.cell_data["Normal"] = rotation_norm.vector.array
warped.set_active_scalars("Normal")
pl.add_mesh(warped, **dargs)
pl.add_axes(color="black")
pl.add_text("Normalized rotation", color='k')
pl.subplot(0, 1)
warped.cell_data["RX"] = rotation_x.vector.array
warped.set_active_scalars("RX")
pl.add_mesh(warped, **dargs)
pl.add_axes(color="black")
pl.add_text("rotation RX", color='k')
pl.subplot(1, 0)
warped.cell_data["RY"] = rotation_y.vector.array
warped.set_active_scalars("RY")
pl.add_mesh(warped, **dargs)
pl.add_axes(color="black")
pl.add_text("rotation RY", color='k')
pl.subplot(1, 1)
warped.cell_data["RZ"] = rotation_z.vector.array
warped.set_active_scalars("RZ")
pl.add_mesh(warped, **dargs)
pl.add_axes(color="black")
pl.add_text("rotation RZ", color='k')

# Lier les vues pour une vue cohérente
pl.link_views()
pl.camera_position = 'iso'
pl.background_color = 'grey'
# Afficher le tout
pl.show()

Widget(value='<iframe src="http://localhost:51527/index.html?ui=P_0x2b91ab440_7&reconnect=auto" class="pyvista…

In [20]:
# Extraction de la valeur maximale RZ
max_rotation_z = np.max(rotation_z.x.array)
print(max_rotation_z)

19.888687722452723


In [21]:
J=pi * (R ** 4) / 2
theta = C*h/(mu*J)
print(theta)

20.0


In [ ]:
# Extraction de la valeur maximale RZ
max_rotation_norm = np.max(rotation_norm.x.array)
print(max_rotation_norm)

In [ ]:
# Extraction de la valeur maximale RZ
max_rotation_y = np.max(rotation_y.x.array)
print(max_rotation_y)

In [ ]:
# Extraction de la valeur maximale RZ
max_rotation_x = np.max(rotation_x.x.array)
print(max_rotation_x)

In [ ]:
import numpy as np

# Valeur analytique de la rotation maximale (ici, 20 radians)
theta_analytique = 20.0

# Valeur numérique récupérée
theta_numerique = max_rotation_z

# Calcul de l'erreur absolue
erreur_absolue = np.abs(theta_analytique - theta_numerique)

# Calcul de l'erreur relative
erreur_relative = erreur_absolue / np.abs(theta_analytique) * 100

# Affichage des résultats
print(f"Rotation analytique maximale : {theta_analytique} radians")
print(f"Rotation numérique maximale : {theta_numerique} radians")
print(f"Erreur absolue : {erreur_absolue} radians")
print(f"Erreur relative : {erreur_relative} %")


In [ ]:
import pyvista as pv
import numpy as np
import dolfinx
import ufl
from petsc4py import PETSc
from mpi4py import MPI
import matplotlib.pyplot as plt

# Calculer le tenseur de déformation
sig = sigma(uh)

# Extraire les composantes du tenseur de déformation
sig_xx = sig[0, 0]
sig_xy = sig[0, 1]
sig_xz = sig[0, 2]
sig_yy = sig[1, 1]
sig_yz = sig[1, 2]
sig_zz = sig[2, 2]

V_sig = fem.functionspace(domain, ("DG", 0))

sig_xx_expr = fem.Expression(sig_xx, V_sig.element.interpolation_points())
stress_xx = fem.Function(V_sig)
stress_xx.interpolate(sig_xx_expr)

sig_yy_expr = fem.Expression(sig_yy, V_sig.element.interpolation_points())
stress_yy = fem.Function(V_sig)
stress_yy.interpolate(sig_yy_expr)

sig_zz_expr = fem.Expression(sig_zz, V_sig.element.interpolation_points())
stress_zz = fem.Function(V_sig)
stress_zz.interpolate(sig_zz_expr)

sig_yz_expr = fem.Expression(sig_yz, V_sig.element.interpolation_points())
stress_yz = fem.Function(V_sig)
stress_yz.interpolate(sig_yz_expr)

sig_xz_expr = fem.Expression(sig_xz, V_sig.element.interpolation_points())
stress_xz = fem.Function(V_sig)
stress_xz.interpolate(sig_xz_expr)

sig_xy_expr = fem.Expression(sig_xy, V_sig.element.interpolation_points())
stress_xy = fem.Function(V_sig)
stress_xy.interpolate(sig_xy_expr)

pl = pv.Plotter(shape=(2, 3))

dargs = dict(
    cmap="jet",
    show_scalar_bar=False,
)

pl.subplot(0, 0)
warped.cell_data["Sigma_XX"] = stress_xx.vector.array
warped.set_active_scalars("Sigma_XX")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Sigma XX", color='k')

pl.subplot(0, 1)
warped.cell_data["Sigma_YY"] = stress_yy.vector.array
warped.set_active_scalars("Sigma_YY")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Sigma YY", color='k')

pl.subplot(0, 2)
warped.cell_data["Sigma_ZZ"] = stress_zz.vector.array
warped.set_active_scalars("Sigma_ZZ")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Sigma ZZ", color='k')

pl.subplot(1, 0)
warped.cell_data["Sigma_YZ"] = stress_yz.vector.array
warped.set_active_scalars("Sigma_YZ")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Sigma YZ", color='k')

pl.subplot(1, 1)
warped.cell_data["Sigma_XZ"] = stress_xz.vector.array
warped.set_active_scalars("Sigma_XZ")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Sigma XZ", color='k')

pl.subplot(1, 2)
warped.cell_data["Sigma_XY"] = stress_xy.vector.array
warped.set_active_scalars("Sigma_XY")
pl.add_mesh(warped, **dargs)    
pl.add_axes(color="black")
pl.add_text("Sigma XY", color='k')

# Lier les vues pour une vue cohérente
pl.link_views()
pl.camera_position = 'iso'
pl.background_color = 'grey'
# Afficher le tout
pl.show()

In [ ]:
s = sigma(uh) - 1. / 3 * ufl.tr(sigma(uh)) * ufl.Identity(len(uh))
von_Mises = ufl.sqrt(3. / 2 * ufl.inner(s, s))

V_von_mises = fem.functionspace(domain, ("DG", 0))
stress_expr = fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
stresses = fem.Function(V_von_mises)
stresses.interpolate(stress_expr)

In [ ]:
# Créez une nouvelle instance de Plotter
warped.cell_data["VonMises"] = stresses.vector.array
warped.set_active_scalars("VonMises")

p = pv.Plotter()
p.add_mesh(warped, cmap="jet",show_edges=False, scalar_bar_args={
    "title": "Von Mises",
    "title_font_size": 24,
    "label_font_size": 22,
    "shadow": True,
    "italic": True,
    "font_family": "arial",
    "vertical": False
})
p.add_text("Von Mises", font_size=12, color="black", position="upper_edge")

p.add_axes(color="black")
p.set_background("grey")
p.show()

In [ ]:
import pyvista as pv
import numpy as np
import dolfinx
import ufl
from petsc4py import PETSc
from mpi4py import MPI
import matplotlib.pyplot as plt

# Calculer le tenseur de déformation
def epsilon(u):
    return 0.5 * (ufl.grad(u).T + ufl.grad(u))

# Calculer le tenseur de déformation
eps = epsilon(uh)

# Créer un espace de fonctions pour les données projetées
V_eps = V

# Créer une fonction pour stocker les résultats projetés
def create_function_from_expression(eps_component, V):
    func = dolfinx.fem.Function(V)
    expr = dolfinx.fem.Expression(eps_component, V.element.interpolation_points())
    func.interpolate(expr)
    return func

# Extraire les composantes du tenseur de déformation
eps_xx = sig[0, 0]
eps_xy = sig[0, 1]
eps_xz = sig[0, 2]
eps_yy = sig[1, 1]
eps_yz = sig[1, 2]
eps_zz = sig[2, 2]

# Projeter chaque composante du tenseur de déformation dans l'espace de fonctions
components = [eps_xx, eps_xy, eps_xz, eps_yy, eps_yz, eps_zz]
component_names = ['Epsilon_XX', 'Epsilon_XY', 'Epsilon_XZ', 'Epsilon_YY', 'Epsilon_YZ', 'Epsilon_ZZ']

# Créez la grille PyVista
eps_topology, eps_cell_types, eps_geometry = dolfinx.plot.vtk_mesh(domain)
eps_grid = pv.UnstructuredGrid(eps_topology, eps_cell_types, eps_geometry)

# Ajouter chaque composante à la grille PyVista
#for i, (component, name) in enumerate(zip(components, component_names)):
#    component_func = create_function_from_expression(component, V_eps)
#    eps_grid[name] = component_func.x.array

V_von_mises = fem.functionspace(domain, ("DG", 0))
stress_expr = fem.Expression(eps_zz, V_von_mises.element.interpolation_points())
stresses = fem.Function(V_von_mises)
stresses.interpolate(stress_expr)

# Créez une nouvelle instance de Plotter
p = pv.Plotter()
warped.cell_data["VonMises"] = stresses.vector.array
warped.set_active_scalars("VonMises")
p.add_mesh(warped, show_edges=False, scalar_bar_args={
    "title": "Von Mises",
    "title_font_size": 24,
    "label_font_size": 22,
    "shadow": True,
    "italic": True,
    "font_family": "arial",
    "vertical": False
})
p.add_text("Von Mises", font_size=12, color="black", position="upper_edge")

p.add_axes(color="black")
p.set_background("grey")
p.show()

In [ ]:
import pyvista as pv
import dolfinx
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np
import gmsh
import meshio
from dolfinx.io import XDMFFile

import dolfinx.fem as fem
import dolfinx.mesh as mesh
from petsc4py import PETSc

# Initialisation de GMSH
gmsh.initialize()
gmsh.model.add("ellipse")

# Définition des variables
a = 0.2  # Demi-grand axe en mètres
b = 0.1  # Demi-petit axe en mètres
h = 1.0  # Hauteur du cylindre en mètres
lc = 0.02  # Longueur caractéristique des éléments en mètres

# Créer une ellipse dans le plan XY au centre (0, 0, 0)
ellipse = gmsh.model.occ.addEllipse(0, 0, 0, a, b)

# Créer une boucle fermée autour de l'ellipse
curve_loop = gmsh.model.occ.addCurveLoop([ellipse])

# Créer une surface à partir de la boucle
surface = gmsh.model.occ.addPlaneSurface([curve_loop])

# Extruder cette surface pour créer un cylindre elliptique de hauteur h
gmsh.model.occ.extrude([(2, surface)], 0, 0, h)

# Synchronisation de la géométrie
gmsh.model.occ.synchronize()

# Définir les tailles des éléments
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)

# Générer le maillage
gmsh.model.mesh.generate(3)  # 3 pour le maillage en 3D

# Sauvegarder le maillage dans un fichier .msh
gmsh.write("ellipse.msh")

# Finaliser GMSH
gmsh.finalize()

# Lire le fichier .msh avec meshio
msh = meshio.read("ellipse.msh")

# Sauvegarder le maillage en format XDMF
meshio.write("ellipse.xdmf", meshio.Mesh(points=msh.points, cells={"tetra": msh.cells_dict.get("tetra", [])}))

# Lire le fichier .xdmf avec Dolfinx
with XDMFFile(MPI.COMM_WORLD, "ellipse.xdmf", "r") as xdmf_file:
    domain = xdmf_file.read_mesh(name="Grid")
    domain.topology.create_connectivity(domain.topology.dim-1, domain.topology.dim)

# Afficher des informations sur le maillage
print("Maillage créé avec succès.")
print(f"Nombre de sommets : {domain.topology.index_map(domain.topology.dim).size_global}")
print(f"Nombre d'éléments : {domain.topology.index_map(domain.topology.dim-1).size_global}")

In [ ]:
import pyvista as pv
import dolfinx
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np
import gmsh
import meshio
from dolfinx.io import XDMFFile

import dolfinx.fem as fem
import dolfinx.mesh as mesh
from petsc4py import PETSc

# Initialisation de GMSH
gmsh.initialize()
gmsh.model.add("cylindre")

# Définir les paramètres du cylindre
radius = 0.2 # 3 cm en mètres
height = 1  # 18 cm en mètres
lc = 0.1    # Longueur caractéristique des éléments en mètres

# Créer un cylindre
gmsh.model.occ.addCylinder(0, 0, 0, 0, 0, height, radius)

# Synchronisation de la géométrie
gmsh.model.occ.synchronize()

# Définir les tailles des éléments
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)

# Générer le maillage
gmsh.model.mesh.generate(3)  # 3 pour le maillage en 3D

# Sauvegarder le maillage dans un fichier .msh
gmsh.write("cylindre.msh")

# Finaliser GMSH
gmsh.finalize()

# Lire le fichier .msh avec meshio
msh = meshio.read("cylindre.msh")

# Sauvegarder le maillage en format XDMF
meshio.write("cylindre.xdmf", meshio.Mesh(points=msh.points, cells={"tetra": msh.cells_dict.get("tetra", [])}))

# Lire le fichier .xdmf avec Dolfinx
with XDMFFile(MPI.COMM_WORLD, "cylindre.xdmf", "r") as xdmf_file:
    domain = xdmf_file.read_mesh(name="Grid")
    domain.topology.create_connectivity(domain.topology.dim-1, domain.topology.dim)

# Afficher des informations sur le maillage
print("Maillage créé avec succès.")
print(f"Nombre de sommets : {domain.topology.index_map(domain.topology.dim).size_global}")
print(f"Nombre d'éléments : {domain.topology.index_map(domain.topology.dim-1).size_global}")